In [19]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import importlib
from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy


In [40]:
PATH='data/language_model/'

TRN_PATH = 'train'
VAL_PATH = 'val'
TRN = f'{PATH}{TRN_PATH}' 
VAL = f'{PATH}{VAL_PATH}' 

In [22]:
with open(f'{PATH}models/TEXT.pkl', 'rb') as pickle_text:
    TEXT = pickle.load(pickle_text)

In [23]:
spacy_tok = spacy.load('en')

In [75]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

bs=64
bptt=70

FILES = dict(train='train', validation='val', test='val')

md = LanguageModelData.from_text_files(f'{PATH}all/', TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [ ]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

In [92]:
# learner = md.get_model(opt_fn, 200, bptt, n_layers=nl, 
#            dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
learner = md.get_model(opt_fn, 200, bptt, n_layers = nl)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)

In [93]:
learner.fit(3e-3, 1, wds=1e-6)

  0%|          | 0/7608 [00:00<?, ?it/s]

RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generic/THCStorage.cu:58

In [ ]:
learner.fit(3e-3, 3, wds=1e-6, cycle_len=1, cycle_mult=2)

In [ ]:
learner.save_encoder('adam2_enc')

In [ ]:
learner.fit(3e-3, 10, wds=1e-6, cycle_len=5, cycle_save_name='adam3_10')

In [ ]:
learner.save_encoder('adam3_10_enc')

In [ ]:
learner.fit(3e-3, 8, wds=1e-6, cycle_len=10, cycle_save_name='adam3_5')

In [ ]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=20, cycle_save_name='adam3_20')

In [91]:
learner.save_encoder('adam3_20_enc')

In [ ]:
learner.save('adam3_20')
